* I had used pycaret to know which model suits best for our prepared data set, however did not include this in the the notebook as this would significantly increase processing time.
* Will be posting new results after I tune the hyper parameters.
* have used GDP datset and passed it as inputs to the model, additionally have calculated holidays.

In [ ]:
import math
import numpy as np
import pandas as pd

import dateutil.easter as easter

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
from learntools.time_series.style import *  # plot style settings
from learntools.time_series.utils import plot_periodogram, seasonal_plot

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess

from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.deterministic import DeterministicProcess

We will be importing GDP data as well to generate more insights from the data, we will use GDP datasets of 

In [ ]:
original_train_df = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
original_test_df = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')
gdp_df = pd.read_csv('../input/gdp-20152019-finland-norway-and-sweden/GDP_data_2015_to_2019_Finland_Norway_Sweden.csv')

gdp_df.set_index('year', inplace=True)

# The dates are read as strings and must be converted
for df in [original_train_df, original_test_df]:
    df['date'] = pd.to_datetime(df.date)
original_train_df.head(2)

In [ ]:
# keeping a function ready to compute SMAPE
def smape_loss(y_true, y_pred):
    """SMAPE Loss"""
    return np.abs(y_true - y_pred) / (y_true + np.abs(y_pred)) * 200

In [ ]:
# Feature engineering
def engineer(df):
    """Return a new dataframe with the engineered features"""
    
    def get_gdp(row):
        country = 'GDP_' + row.country
        return gdp_df.loc[row.date.year, country]
        
    new_df = pd.DataFrame({'gdp': np.log(df.apply(get_gdp, axis=1)),
                           'wd4': df.date.dt.weekday == 4, # Friday
                           'wd56': df.date.dt.weekday >= 5, # Saturday and Sunday
                          })

    # One-hot encoding (no need to encode the last categories)
    for country in ['Finland', 'Norway']:
        new_df[country] = df.country == country
    new_df['KaggleRama'] = df.store == 'KaggleRama'
    for product in ['Kaggle Mug', 'Kaggle Hat']:
        new_df[product] = df['product'] == product
        
    # Seasonal variations (Fourier series)
    # The three products have different seasonal patterns
    dayofyear = df.date.dt.dayofyear
    for k in range(1, 3):
        new_df[f'sin{k}'] = np.sin(dayofyear / 365 * 2 * math.pi * k)
        new_df[f'cos{k}'] = np.cos(dayofyear / 365 * 2 * math.pi * k)
        new_df[f'mug_sin{k}'] = new_df[f'sin{k}'] * new_df['Kaggle Mug']
        new_df[f'mug_cos{k}'] = new_df[f'cos{k}'] * new_df['Kaggle Mug']
        new_df[f'hat_sin{k}'] = new_df[f'sin{k}'] * new_df['Kaggle Hat']
        new_df[f'hat_cos{k}'] = new_df[f'cos{k}'] * new_df['Kaggle Hat']

    return new_df

train_df = engineer(original_train_df)
train_df['date'] = original_train_df.date
train_df['num_sold'] = original_train_df.num_sold.astype(np.float32)
test_df = engineer(original_test_df)

features = test_df.columns

for df in [train_df, test_df]:
    df[features] = df[features].astype(np.float32)
print(list(features))

In [ ]:
# Feature engineering for holidays
def engineer_more(df):
    """Return a new dataframe with more engineered features"""
    new_df = engineer(df)

    # End of year
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d)
                                      for d in range(24, 32)}),
                        pd.DataFrame({f"n-dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d) & (df.country == 'Norway')
                                      for d in range(24, 32)}),
                        pd.DataFrame({f"f-jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Finland')
                                      for d in range(1, 14)}),
                        pd.DataFrame({f"jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Norway')
                                      for d in range(1, 10)}),
                        pd.DataFrame({f"s-jan{d}":
                                      (df.date.dt.month == 1) & (df.date.dt.day == d) & (df.country == 'Sweden')
                                      for d in range(1, 15)})],
                       axis=1)
    
    # May
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) 
                                      for d in list(range(1, 10))}), #  + list(range(17, 25))
                        pd.DataFrame({f"may{d}":
                                      (df.date.dt.month == 5) & (df.date.dt.day == d) & (df.country == 'Norway')
                                      for d in list(range(19, 26))})],
                       axis=1)
    
    # June and July
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"june{d}":
                                      (df.date.dt.month == 6) & (df.date.dt.day == d) & (df.country == 'Sweden')
                                      for d in list(range(8, 14))}),
                        #pd.DataFrame({f"june{d}":
                        #              (df.date.dt.month == 6) & (df.date.dt.day == d) & (df.country == 'Norway')
                        #              for d in list(range(22, 31))}),
                        #pd.DataFrame({f"july{d}":
                        #              (df.date.dt.month == 7) & (df.date.dt.day == d) & (df.country == 'Norway')
                        #              for d in list(range(1, 3))})],
                       ],
                       axis=1)
    
    # Last Wednesday of June
    wed_june_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-06-24')),
                                         2016: pd.Timestamp(('2016-06-29')),
                                         2017: pd.Timestamp(('2017-06-28')),
                                         2018: pd.Timestamp(('2018-06-27')),
                                         2019: pd.Timestamp(('2019-06-26'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"wed_june{d}": 
                                      (df.date - wed_june_date == np.timedelta64(d, "D")) & (df.country != 'Norway')
                                      for d in list(range(-4, 6))})],
                       axis=1)
    
    # First Sunday of November
    sun_nov_date = df.date.dt.year.map({2015: pd.Timestamp(('2015-11-1')),
                                         2016: pd.Timestamp(('2016-11-6')),
                                         2017: pd.Timestamp(('2017-11-5')),
                                         2018: pd.Timestamp(('2018-11-4')),
                                         2019: pd.Timestamp(('2019-11-3'))})
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"sun_nov{d}": 
                                      (df.date - sun_nov_date == np.timedelta64(d, "D")) & (df.country != 'Norway')
                                      for d in list(range(0, 9))})],
                       axis=1)
    
    # First half of December (Independence Day of Finland, 6th of December)
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"dec{d}":
                                      (df.date.dt.month == 12) & (df.date.dt.day == d) & (df.country == 'Finland')
                                      for d in list(range(6, 14))})],
                       axis=1)

    # Easter
    easter_date = df.date.apply(lambda date: pd.Timestamp(easter.easter(date.year)))
    new_df = pd.concat([new_df,
                        pd.DataFrame({f"easter{d}": 
                                      (df.date - easter_date == np.timedelta64(d, "D"))
                                      for d in list(range(-2, 11)) + list(range(40, 48)) + list(range(50, 59))})],
                       axis=1)
    
    return new_df.astype(np.float32)

train_df = engineer_more(original_train_df)
train_df['date'] = original_train_df.date
train_df['num_sold'] = original_train_df.num_sold.astype(np.float32)
test_df = engineer_more(original_test_df)

features = list(test_df.columns)
print(list(features))

In [ ]:
from catboost import CatBoostRegressor
import catboost
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
train_df.drop('date', axis=1, inplace=True)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(train_df.drop('num_sold',axis=1),train_df['num_sold'], test_size=0.2)
ct = CatBoostRegressor(verbose=False, eval_metric='SMAPE')
ct.fit(X_train, y_train, eval_set=(X_val,y_val))
ct

In [ ]:
pred = ct.predict(test_df)
pred = [int(i) for i in pred] 
# new_test_df = pd.read_csv("../input/tabular-playground-series-jan-2022/test.csv")
# new_test_df['num_sold'] = pred
# new_test_df = new_test_df[['row_id','num_sold']]
# new_test_df.to_csv('submission.csv',index=False)

### This notebook will be updated further.

### Let us try creating different models for different stores

In [ ]:
original_train_df["store"].value_counts()

In [ ]:
train_mart = original_train_df[original_train_df["store"]=='KaggleMart']
train_rama = original_train_df[original_train_df["store"]=='KaggleRama']
test_mart = original_test_df[original_test_df["store"]=='KaggleMart']
test_rama = original_test_df[original_test_df["store"]=='KaggleRama']

In [ ]:
original_train_df.head()

In [ ]:
test_mart

In [ ]:
#mart data prep
train_df_mart = engineer_more(train_mart)
train_df_mart['date'] = train_mart.date
train_df_mart['num_sold'] = train_mart.num_sold.astype(np.float32)
test_df_mart = engineer_more(test_mart)

features_mart = list(test_df_mart.columns)

#rama data prep
train_df_rama = engineer_more(train_rama)
train_df_rama['date'] = train_rama.date
train_df_rama['num_sold'] = train_rama.num_sold.astype(np.float32)
test_df_rama = engineer_more(test_rama)

features_rama = list(test_df_rama.columns)

In [ ]:
# mart model and predictions
X_train_mart, X_val_mart, y_train_mart, y_val_mart = train_test_split(train_df_mart.drop('num_sold',axis=1),train_df_mart['num_sold'], test_size=0.1)
ct_mart = CatBoostRegressor(verbose=False, eval_metric='SMAPE')
ct_mart.fit(X_train_mart, y_train_mart, eval_set=(X_val_mart,y_val_mart))

pred_mart = ct.predict(test_df_mart)
pred_mart = [int(i) for i in pred_mart] 
test_mart['num_sold'] = pred_mart
test_mart = test_mart[['row_id','num_sold']]

# rama model and predictions
X_train_rama, X_val_rama, y_train_rama, y_val_rama = train_test_split(train_df_rama.drop('num_sold',axis=1),train_df_rama['num_sold'], test_size=0.1)
ct_rama = CatBoostRegressor(verbose=False, eval_metric='SMAPE')
ct_rama.fit(X_train_rama, y_train_rama, eval_set=(X_val_rama,y_val_rama))

pred_rama = ct.predict(test_df_rama)
pred_rama = [int(i) for i in pred_rama] 
test_rama['num_sold'] = pred_rama
test_rama = test_rama[['row_id','num_sold']]

#submission
submission = pd.concat([test_mart,test_rama])
submission.sort_index(inplace=True)
submission.to_csv('submission.csv',index=False)

In [ ]:
submission